# Load libs

In [2]:
import os
import requests

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd
import json
import datetime
import re

pd.set_option('display.max_columns', None)

# Load .env & handle

In [7]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [8]:
HANDLE = "@tuki.music_official"

## 1. Lấy thông tin channel + uploads playlist

In [9]:
ch_res = requests.get(
    "https://www.googleapis.com/youtube/v3/channels",
    params={
        "part": "snippet, statistics, contentDetails",
        "forHandle": HANDLE,
        "key": API_KEY
    }
).json()

if not ch_res.get("items"):
    raise SystemExit("Không tìm thấy channel với handle này.")

ch_item = ch_res["items"][0]
channel_id = ch_item["id"]
upload_pl = ch_item["contentDetails"]["relatedPlaylists"]["uploads"]

print("Channel ID:", channel_id)
print("Title:", ch_item["snippet"]["title"])
print("Subs:", ch_item["statistics"].get("subscriberCount"))

Channel ID: UCp74ruKv2sUiI2HqEhH8pRw
Title: tuki.(16)
Subs: 1100000


In [4]:
ch_res

{'kind': 'youtube#channelListResponse',
 'etag': 'gmB4e6iGPQXGYLe40Yrou1HXSa0',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '-Krf-eCpt5_PDvenghECPP5-vDY',
   'id': 'UCp74ruKv2sUiI2HqEhH8pRw',
   'snippet': {'title': 'tuki.(16)',
    'description': '高校二年生 16歳 弾き語り 誰かの心に響きますように。',
    'customUrl': '@tuki.music_official',
    'publishedAt': '2023-07-18T07:32:30.72368Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s800-c-k-c0x00ffffff-no-rj',
      'width': 800,
     

In [10]:
df_channel_dbt = pd.json_normalize(ch_res, sep="__")
df_channel_dbt

,kind,etag,items,pageInfo__totalResults,pageInfo__resultsPerPage
0,youtube#channelListResponse,gmB4e6iGPQXGYLe40Yrou1HXSa0,"[{'kind': 'youtube#channel', 'etag': '-Krf-eCp...",1,5


In [49]:
# Trích xuất dữ liệu
channels = []
for item in ch_res.get("items", []):
    channels.append({
        "channel_id": item["id"],
        "title": item["snippet"]["title"],
        "description": item["snippet"]["description"],
        "customUrl": item["snippet"].get("customUrl"),
        "publishedAt": item["snippet"]["publishedAt"],
        "country": item["snippet"].get("country"),
        "thumbnail_high": item["snippet"]["thumbnails"]["high"]["url"],
        "uploads_playlist": item["contentDetails"]["relatedPlaylists"]["uploads"],
        "subscriberCount": int(item["statistics"].get("subscriberCount", 0)),
        "viewCount": int(item["statistics"].get("viewCount", 0)),
        "videoCount": int(item["statistics"].get("videoCount", 0)),
    })

# Đưa vào DataFrame
df_channels = pd.DataFrame(channels)
df_channels

,channel_id,title,description,customUrl,publishedAt,country,thumbnail_high,uploads_playlist,subscriberCount,viewCount,videoCount
0,UCp74ruKv2sUiI2HqEhH8pRw,tuki.(16),高校二年生 16歳 弾き語り 誰かの心に響きますように。,@tuki.music_official,2023-07-18T07:32:30.72368Z,JP,https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFS...,UUp74ruKv2sUiI2HqEhH8pRw,1100000,552984374,330


## 2. Lấy ra playlists

In [12]:
# Lấy ra toàn bộ playlists của channel
pl_items = []
page_token = None

while True:
    pl_res = requests.get(
        "https://www.googleapis.com/youtube/v3/playlists",
        params={
            "part": "snippet,contentDetails",
            "channelId": channel_id,
            "maxResults": 50,
            "pageToken": page_token,
            "key": API_KEY
        }
    ).json()

    pl_items.extend(pl_res.get("items", []))
    page_token = pl_res.get("nextPageToken")
    if not page_token:
        break

print("Số playlists:", len(pl_items))

Số playlists: 1


In [13]:
pl_items[0]

{'kind': 'youtube#playlist',
 'etag': 'VpEo9bH9SdxqLdWVmR9rcJahWo0',
 'id': 'PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA',
 'snippet': {'publishedAt': '2023-12-28T10:55:13.295457Z',
  'channelId': 'UCp74ruKv2sUiI2HqEhH8pRw',
  'title': 'ミュージックビデオ',
  'description': '',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg',
    'width': 480,
    'height': 360},
   'standard': {'url': 'https://i.ytimg.com/vi/0-MocwNWSPU/sddefault.jpg',
    'width': 640,
    'height': 480},
   'maxres': {'url': 'https://i.ytimg.com/vi/0-MocwNWSPU/maxresdefault.jpg',
    'width': 1280,
    'height': 720}},
  'channelTitle': 'tuki.(16)',
  'localized': {'title': 'ミュージックビデオ', 'description': ''}},
 'contentDetails': {'itemCount': 18}}

In [16]:
df_playlist_dbt = pd.json_normalize(pl_res, sep="__")
df_playlist_dbt.head(1)

,kind,etag,items,pageInfo__totalResults,pageInfo__resultsPerPage
0,youtube#playlistListResponse,Bys6gZ3JLTB8e4ymGdq-4uwndaA,"[{'kind': 'youtube#playlist', 'etag': 'VpEo9bH...",1,50


In [15]:
df_channel_dbt['items'][0]

[{'kind': 'youtube#channel',
  'etag': '-Krf-eCpt5_PDvenghECPP5-vDY',
  'id': 'UCp74ruKv2sUiI2HqEhH8pRw',
  'snippet': {'title': 'tuki.(16)',
   'description': '高校二年生 16歳 弾き語り 誰かの心に響きますように。',
   'customUrl': '@tuki.music_official',
   'publishedAt': '2023-07-18T07:32:30.72368Z',
   'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s88-c-k-c0x00ffffff-no-rj',
     'width': 88,
     'height': 88},
    'medium': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s240-c-k-c0x00ffffff-no-rj',
     'width': 240,
     'height': 240},
    'high': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s800-c-k-c0x00ffffff-no-rj',
     'width': 800,
     'height': 800}},
   'localized': {'title': 'tuki.(16)',
    'description': '高校二年生 16歳 弾き語り 誰かの心に響きますように。'},
   'country': 'JP'},
  'contentDetails': {'relatedP

In [19]:
# phẳng JSON -> DataFrame
df = pd.json_normalize(pl_items, sep="__")

# chuẩn hóa tên cột chính
rename_map = {
    "id": "playlist_id",
    "snippet__channelId": "channel_id",
    "snippet__channelTitle": "channel_title",
    "snippet__title": "playlist_title",
    "snippet__description": "playlist_desc",
    "snippet__publishedAt": "published_at",
    "contentDetails__itemCount": "item_count",
    "snippet__thumbnails__default__url": "thumb_default",
    "snippet__thumbnails__medium__url": "thumb_medium",
    "snippet__thumbnails__high__url": "thumb_high",
}
df = df.rename(columns=rename_map)

# ép kiểu & parse thời gian
df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")
df["item_count"]   = pd.to_numeric(df["item_count"], errors="coerce").astype("Int64")

# chọn subset cột gọn gàng
df_playlists = df[
    ["playlist_id", "playlist_title", "playlist_desc",
     "published_at", "channel_id", "channel_title",
     "item_count", "thumb_default", "thumb_medium", "thumb_high"]
]

df_playlists.head(1)

,playlist_id,playlist_title,playlist_desc,published_at,channel_id,channel_title,item_count,thumb_default,thumb_medium,thumb_high
0,PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,ミュージックビデオ,,2023-12-28 10:55:13.295457+00:00,UCp74ruKv2sUiI2HqEhH8pRw,tuki.(16),18,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg


## 3. Lấy ra Videos trong Playlist

In [20]:
channel_id

'UCp74ruKv2sUiI2HqEhH8pRw'

In [21]:
# Lấy items (videos) trong từng playlist
def fetch_playlist_items(playlist_id):
    items, page_token = [], None
    while True:
        res = requests.get(
            "https://www.googleapis.com/youtube/v3/playlistItems",
            params={
                "part": "snippet,contentDetails,status",
                "playlistId": playlist_id,
                "maxResults": 50,
                "pageToken": page_token,
                "key": API_KEY
            },
            timeout=30
        ).json()
        items.extend(res.get("items", []))
        page_token = res.get("nextPageToken")
        if not page_token:
            break
    return items

all_items = []
for pid in df_playlists["playlist_id"]:
    items = fetch_playlist_items(pid)
    for it in items:
        it["playlist_id"] = pid  #thêm id playlist cha để join
    all_items.extend(items)

print("Tổng số video trong tất cả playlists:", len(all_items))

df_playlist_items = pd.json_normalize(all_items, sep="__").rename(columns={"id": "playlistItem_id"})
df_playlist_items = df_playlist_items.rename(columns={
    "snippet__title": "video_title",
    "snippet__description": "video_desc",
    "snippet__publishedAt": "added_to_playlist_at",
    "contentDetails__videoPublishedAt": "video_published_at",
    "contentDetails__videoId": "video_id",
    "snippet__channelId": "channel_id",
    "snippet__channelTitle": "channel_title",
    "snippet__position": "position_in_playlist",
})
df_playlist_items["added_to_playlist_at"] = pd.to_datetime(df_playlist_items["added_to_playlist_at"], errors="coerce")
df_playlist_items["video_published_at"] = pd.to_datetime(df_playlist_items["video_published_at"], errors="coerce")

# --- Kết quả ---
print("Playlists shape:", df_playlists.shape)
print("Items shape:", df_playlist_items.shape)

Tổng số video trong tất cả playlists: 18
Playlists shape: (1, 10)
Items shape: (18, 33)


In [22]:
df_playlist_items.head(1)

,kind,etag,playlistItem_id,playlist_id,added_to_playlist_at,channel_id,video_title,video_desc,snippet__thumbnails__default__url,snippet__thumbnails__default__width,snippet__thumbnails__default__height,snippet__thumbnails__medium__url,snippet__thumbnails__medium__width,snippet__thumbnails__medium__height,snippet__thumbnails__high__url,snippet__thumbnails__high__width,snippet__thumbnails__high__height,snippet__thumbnails__standard__url,snippet__thumbnails__standard__width,snippet__thumbnails__standard__height,snippet__thumbnails__maxres__url,snippet__thumbnails__maxres__width,snippet__thumbnails__maxres__height,channel_title,snippet__playlistId,position_in_playlist,snippet__resourceId__kind,snippet__resourceId__videoId,snippet__videoOwnerChannelTitle,snippet__videoOwnerChannelId,video_id,video_published_at,status__privacyStatus
0,youtube#playlistItem,khiHaRIORRz8m2SdKnzXjIAyl6s,UExzMHZVMU9GOHJwb1ZSRE5VMVBBYUc3WTFRZXZHRERNQS...,PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,2025-08-31 14:01:24+00:00,UCp74ruKv2sUiI2HqEhH8pRw,tuki.『最低界隈』Official Lyric Video,tuki.『最低界隈』Streaming & Download\nhttps://lnk.t...,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,120,90,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg,480,360,https://i.ytimg.com/vi/0-MocwNWSPU/sddefault.jpg,640,480,https://i.ytimg.com/vi/0-MocwNWSPU/maxresdefau...,1280,720,tuki.(16),PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,0,youtube#video,0-MocwNWSPU,tuki.(16),UCp74ruKv2sUiI2HqEhH8pRw,0-MocwNWSPU,2025-08-31 13:17:06+00:00,public


## 4. Lấy statistics của các videos

In [23]:
# Lấy ra danh sách video_id duy nhất
list_video = df_playlist_items["video_id"].dropna().astype(str).unique().tolist()
print("Số video duy nhất:", len(list_video))

Số video duy nhất: 18


In [25]:
#Hàm gọi videos.list để lấy statistics (batch 50 ID/lần)
def fetch_videos_stats(video_ids, api_key, part="snippet,contentDetails,statistics,liveStreamingDetails"):
    all_items = []
    for i in range(0, len(video_ids), 50):
        batch_ids = ",".join(video_ids[i:i+50])
        res = requests.get(
            "https://www.googleapis.com/youtube/v3/videos",
            params={"part": part, "id": batch_ids, "key": api_key},
            timeout=30
        )
        res.raise_for_status()
        all_items.extend(res.json().get("items", []))
    if not all_items:
        return pd.DataFrame(columns=[
            "video_id","video_title","video_desc","video_published_at",
            "duration_iso8601","view_count","like_count","comment_count",
            "v_thumb_default","v_thumb_medium","v_thumb_high",
            "live_actual_start","live_actual_end","live_scheduled_start"
        ])

    df = pd.json_normalize(all_items, sep="__").rename(columns={"id": "video_id"})
    # chọn & chuẩn hoá cột
    rename_map = {
        "snippet__title": "video_title",
        "snippet__description": "video_desc",
        "snippet__publishedAt": "video_published_at",
        "contentDetails__duration": "duration_iso8601",
        "statistics__viewCount": "view_count",
        "statistics__likeCount": "like_count",
        "statistics__commentCount": "comment_count",
        "snippet__thumbnails__default__url": "v_thumb_default",
        "snippet__thumbnails__medium__url": "v_thumb_medium",
        "snippet__thumbnails__high__url": "v_thumb_high",
        "liveStreamingDetails__actualStartTime": "live_actual_start",
        "liveStreamingDetails__actualEndTime": "live_actual_end",
        "liveStreamingDetails__scheduledStartTime": "live_scheduled_start",
    }
    df = df.rename(columns=rename_map)

    # ép kiểu số
    for c in ["view_count","like_count","comment_count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

    # parse thời gian
    for c in ["video_published_at","live_actual_start","live_actual_end","live_scheduled_start"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)

    # đảm bảo đủ cột
    for col in ["video_id","video_title","video_desc","video_published_at",
                "duration_iso8601","view_count","like_count","comment_count",
                "v_thumb_default","v_thumb_medium","v_thumb_high",
                "live_actual_start","live_actual_end","live_scheduled_start"]:
        if col not in df.columns:
            df[col] = pd.NA

    return df[[
        "video_id","video_title","video_desc","video_published_at",
        "duration_iso8601","view_count","like_count","comment_count",
        "v_thumb_default","v_thumb_medium","v_thumb_high",
        "live_actual_start","live_actual_end","live_scheduled_start"
    ]]

In [26]:
all_items

[{'kind': 'youtube#playlistItem',
  'etag': 'khiHaRIORRz8m2SdKnzXjIAyl6s',
  'id': 'UExzMHZVMU9GOHJwb1ZSRE5VMVBBYUc3WTFRZXZHRERNQS4yMDhBMkNBNjRDMjQxQTg1',
  'snippet': {'publishedAt': '2025-08-31T14:01:24Z',
   'channelId': 'UCp74ruKv2sUiI2HqEhH8pRw',
   'title': 'tuki.『最低界隈』Official Lyric Video',
   'description': 'tuki.『最低界隈』Streaming & Download\nhttps://lnk.to/SaiteiKaiwai_\n\n世界で話題の、日本発のアーティストをYouTube Musicの「JAPANIQUE」プレイリストで聴こう:\nhttps://music.youtube.com/playlist?list=RDCLAK5uy_lN7cCzqFOTV83PeDYGf_6N0DiMF4Pnl2E\n\n『最低界隈』作詞・作曲：tuki\n\n満員電車触んなって\nきしょすぎな意味わからんおっさん\n制服スカート寒さ我慢\n生きにくさですら慣れる練習？\nアガらないね終わってんね\n給料バイトよりちょいマシ？\n人手が欲しい少子化だし\n育てる未来見えんのこっち\n\n日々日々困惑\n矛盾に不純に横着\n不正に税金めにめに\n夢見ろって？現実はバグ\n推しのライブいつも当たらんのエグい\ngimme gimme 当落BeRealで報告\n\nWi-Fiあっても心の中圏外\n笑顔の下ではずっと不在通知\nうざい無理\n歪な世界だ\n将来いつかはママになってみたい\nとかなんとか思ってたっけ\nアプデ待ちの世の中\nどうにかして\n\n最低界隈最低界隈\nあいどんのー本当\n奥歯がたがた言わしたろかい\n最低界隈最低界隈\n夢は何？ラブみラブみ 『奪われない事』\nテステスマイクテスト 私絶対的に私です\nSNSでレス\n誰のせいでもないのは嘘です\nテステスマイクテスト\n私絶対的に私です\nSNSでレス\nさささ最低界隈です\n\nお

In [27]:
# Gọi API và merge vào df_items
df_video_stats = fetch_videos_stats(list_video, API_KEY)
print("df_video_stats shape", df_video_stats.shape)

df_video_stats shape (18, 14)


In [28]:
df_video_stats.head(2)

,video_id,video_title,video_desc,video_published_at,duration_iso8601,view_count,like_count,comment_count,v_thumb_default,v_thumb_medium,v_thumb_high,live_actual_start,live_actual_end,live_scheduled_start
0,0-MocwNWSPU,tuki.『最低界隈』Official Lyric Video,tuki.『最低界隈』Streaming & Download\nhttps://lnk.t...,2025-08-31 13:17:06+00:00,PT3M20S,337954,11490,267,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg,2025-08-31 13:17:06+00:00,2025-08-31 13:21:27+00:00,2025-08-31 13:17:00+00:00
1,agstsXgsxg4,tuki.『孤独の鯨』Official Music Video,tuki. 1st Album「15」\nCD：https://tuki.lnk.to/20...,2025-08-31 13:21:06+00:00,PT3M37S,392855,15788,390,https://i.ytimg.com/vi/agstsXgsxg4/default.jpg,https://i.ytimg.com/vi/agstsXgsxg4/mqdefault.jpg,https://i.ytimg.com/vi/agstsXgsxg4/hqdefault.jpg,2025-08-31 13:21:06+00:00,2025-08-31 13:26:08+00:00,2025-08-31 13:21:00+00:00


# Tổng hợp data

In [46]:
df_playlist_items.head(2)

,kind,etag,playlistItem_id,playlist_id,added_to_playlist_at,channel_id,video_title,video_desc,snippet__thumbnails__default__url,snippet__thumbnails__default__width,snippet__thumbnails__default__height,snippet__thumbnails__medium__url,snippet__thumbnails__medium__width,snippet__thumbnails__medium__height,snippet__thumbnails__high__url,snippet__thumbnails__high__width,snippet__thumbnails__high__height,snippet__thumbnails__standard__url,snippet__thumbnails__standard__width,snippet__thumbnails__standard__height,snippet__thumbnails__maxres__url,snippet__thumbnails__maxres__width,snippet__thumbnails__maxres__height,channel_title,snippet__playlistId,position_in_playlist,snippet__resourceId__kind,snippet__resourceId__videoId,snippet__videoOwnerChannelTitle,snippet__videoOwnerChannelId,video_id,video_published_at,status__privacyStatus
0,youtube#playlistItem,khiHaRIORRz8m2SdKnzXjIAyl6s,UExzMHZVMU9GOHJwb1ZSRE5VMVBBYUc3WTFRZXZHRERNQS...,PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,2025-08-31 14:01:24+00:00,UCp74ruKv2sUiI2HqEhH8pRw,tuki.『最低界隈』Official Lyric Video,tuki.『最低界隈』Streaming & Download\nhttps://lnk.t...,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,120,90,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg,480,360,https://i.ytimg.com/vi/0-MocwNWSPU/sddefault.jpg,640,480,https://i.ytimg.com/vi/0-MocwNWSPU/maxresdefau...,1280,720,tuki.(16),PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,0,youtube#video,0-MocwNWSPU,tuki.(16),UCp74ruKv2sUiI2HqEhH8pRw,0-MocwNWSPU,2025-08-31 13:17:06+00:00,public
1,youtube#playlistItem,XdW1Pk_bLvHRKDxFgMPStPXPPiU,UExzMHZVMU9GOHJwb1ZSRE5VMVBBYUc3WTFRZXZHRERNQS...,PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,2025-08-31 14:01:15+00:00,UCp74ruKv2sUiI2HqEhH8pRw,tuki.『孤独の鯨』Official Music Video,tuki. 1st Album「15」\nCD：https://tuki.lnk.to/20...,https://i.ytimg.com/vi/agstsXgsxg4/default.jpg,120,90,https://i.ytimg.com/vi/agstsXgsxg4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/agstsXgsxg4/hqdefault.jpg,480,360,https://i.ytimg.com/vi/agstsXgsxg4/sddefault.jpg,640,480,https://i.ytimg.com/vi/agstsXgsxg4/maxresdefau...,1280,720,tuki.(16),PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,1,youtube#video,agstsXgsxg4,tuki.(16),UCp74ruKv2sUiI2HqEhH8pRw,agstsXgsxg4,2025-08-31 13:21:06+00:00,public


In [37]:
df_playlists.head(2)

,playlist_id,playlist_title,playlist_desc,published_at,channel_id,channel_title,item_count,thumb_default,thumb_medium,thumb_high
0,PLs0vU1OF8rpoVRDNU1PAaG7Y1QevGDDMA,ミュージックビデオ,,2023-12-28 10:55:13.295457+00:00,UCp74ruKv2sUiI2HqEhH8pRw,tuki.(16),18,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg


In [47]:
df_video_stats.head(2)

,video_id,video_title,video_desc,video_published_at,duration_iso8601,view_count,like_count,comment_count,v_thumb_default,v_thumb_medium,v_thumb_high,live_actual_start,live_actual_end,live_scheduled_start
0,0-MocwNWSPU,tuki.『最低界隈』Official Lyric Video,tuki.『最低界隈』Streaming & Download\nhttps://lnk.t...,2025-08-31 13:17:06+00:00,PT3M20S,337954,11490,267,https://i.ytimg.com/vi/0-MocwNWSPU/default.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/mqdefault.jpg,https://i.ytimg.com/vi/0-MocwNWSPU/hqdefault.jpg,2025-08-31 13:17:06+00:00,2025-08-31 13:21:27+00:00,2025-08-31 13:17:00+00:00
1,agstsXgsxg4,tuki.『孤独の鯨』Official Music Video,tuki. 1st Album「15」\nCD：https://tuki.lnk.to/20...,2025-08-31 13:21:06+00:00,PT3M37S,392855,15788,390,https://i.ytimg.com/vi/agstsXgsxg4/default.jpg,https://i.ytimg.com/vi/agstsXgsxg4/mqdefault.jpg,https://i.ytimg.com/vi/agstsXgsxg4/hqdefault.jpg,2025-08-31 13:21:06+00:00,2025-08-31 13:26:08+00:00,2025-08-31 13:21:00+00:00


# Save data

In [51]:
df_channels.to_csv("data/channels.csv", index=False, encoding='utf-8-sig')
df_playlists.to_csv("data/playlists.csv", index=False, encoding='utf-8-sig')
df_playlist_items.to_csv("data/playlist_items.csv", index=False, encoding='utf-8-sig')
df_video_stats.to_csv("data/video_stats.csv", index=False, encoding='utf-8-sig')